In [ ]:
LAB_USER_NAME = 'DEMO'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar

In [ ]:
import panel as pn
pn.extension()
import sys
import os

# add inline dashboard libraries to path so they can be imported later
sys.path.append('../../../dashboards-inline')

LAB_DIR = os.path.join('/home/data/', LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('Data will be saved to', LAB_DIR)

## Part One: The MRI Prescan

### 1. Background Theory on RF Probes
The $B_1$ field is generated by an RF coil in parallel with a capacitor which functions as a simple RLC resonator circuit. The resonant frequency of an RLC circuit is as follows:

$$f_0 = \frac{1}{2\pi\sqrt{LC}}$$

Adjusting the *Tune* capacitor on the probe changes the value of $C$. The RLC resonator circuit also needs to be impedance matched to the RF transmit amplifier and receive preamplifier, which have a characteristic impedance of 50 Ohms. This is accomplished with a *Match* capacitor.

The Wobble tool plots the $S_{11}$ curve of the RF coil, which indicates the tune and match of the RF coil. An $S_{11}$ plots the reflected signal of the the coil as a function of frequency. The position of the dip in the $S_{11}$ curve is mostly controlled by the *Tune* capacitor, while the depth of the dip is mostly controlled by the *Match* capacitor, though there is some overlap in effect. Power will be transfered to the coil most efficiently when the amount of reflected power measured by $S_{11}$ is minimised.

### 2. Setting up the System Variables
#### **Step One: Calculating the Larmor Frequency**
Calculate the desired frequency of the system knowning that $^1\mathrm{H}$ nucleus has a gyromagnetic ratio of 42.576 MHz/T and a system field strength of about 0.3425 T. Include the calculation in your lab report. Enter your calculated value in Hz below (scientific notation will work, e.g. "5.5e6"):

In [ ]:
frequency = pn.widgets.FloatInput(name='Frequency (Hz)', start=0, step=1000)
frequency

#### **Step Two: Tuning the Probe**
Using the **Wobble** tool below in loop mode, adjust the variable capacitors with the provided screwdriver to tune and match the RF coil to your calculated Larmor frequency (Note: the dotted line on the plot corresponds to the desired frequency). Include the dB measurement of your coil tune - this will require a simple calculation. Can also use this step to show how "loading" a coil will affect both the tune and match of the coil.



In [ ]:
from Wobble import WobbleApp # from dashboards-inline directory that was added to sys.path
wobble = WobbleApp(
    override_pars=dict(
        f=frequency, # use value of fequency from the user input
        f_bw=2e6 # fix bandwidth at 2 MHz
    )
)
wobble.main()

NOTE: If using **Run Loop**, click abort before continuing

#### **Step Three: Measuring the Larmor Frequency**
First ensure the **shim sample** is loaded at the correct depth. Then run the **FID** tool below, with your calulated frequency and a starting RF amplitude guess of $0.5$ (half of maximum). The spectrum plot on the right side displays the FFT for the FID signal. The x-axis displays the relative frequency of the signal - where 0 Hz is the $f_0$ of the RF pulse. Does your acquired signal peak correspond to your calculated Larmor frequency - why or why not?

Measure the relative frequency offset of the **Spectrum** peak (try the zoom tools at the top right of the plot) and add it to your calculated frequency, then enter the new value into the frequency input and rerun the **FID** tool to check the spectrum peak is now centered. Record this value for your report.

Open the **Find Frequency** dashboard from the **Dashboard Apps** sidebar and run it. How does the frequency you found compare?

Now that the correct frequency has been measured, go back to **Step Two**, rerun wobble, and retune the probe if necessary.

In [ ]:
rf_amp = pn.widgets.FloatInput(name='RF Amplitude (fraction of maximum)', start=0, end=1, step=0.1, value=0.5)
# Also display frequency again, as FID uses it
pn.Row(frequency, rf_amp)

In [ ]:
from FID import FIDApp # from dashboards-inline directory that was added to sys.path

# setup user shims file
import yaml
SHIM_FILE = os.path.join(LAB_DIR, 'shims.yaml')
DEFAULT_SHIMS = dict(
    shim_x=0,
    shim_y=0,
    shim_z=0,
    shim_z2=0,
    shim_zx=0,
    shim_zy=0,
    shim_xy=0,
    shim_x2y2=0)
if not os.path.isfile(SHIM_FILE):
    with open(SHIM_FILE, 'w') as f:
        yaml.dump(DEFAULT_SHIMS, f)

# create FID experiment using user shims, frequency, and RF amplitude
fid = FIDApp(
    override_par_files=[
        SHIM_FILE
    ],
    override_pars=dict(
        f=frequency, # use value of frequency from the user input
        a_90=rf_amp, # use rf amplitude from the user input
        t_90=32e-6,
        n_scans=1,
        n_samples=10000,
        t_dw=0.5e-6,
        t_acqdelay=50e-6,
        t_end=0.5
    )
)
fid.main()

#### **Step Four: Pulse Calibration**
With the system tuned, it is time to determine the RF pulse amplitude necessary to achieve a $90^\circ$ tip of the magnetization. Try changing the RF Amplitude above and observing the effect on the initial value of the real part of the **Signal** plot. Roughly what RF Amplitude maximises the signal?

Open the **Pulse Calibration** dashboard from the **Dashboard Apps** sidebar and run it. With **Pulse Calibration** dashboard tool the pulse amplitude for a given pulse length is found to maximize signal - giving the pulse size for a $90^\circ$ tip. Make sure *soft pulse* is **deselected** and the *pulse width* is $32\mathrm{\mu s}$. When the pulse calibration is finished, the results will be automatically saved to the system global variables, and the amplitude value can also be read off the $x$ axis at the position of the dashed line (Note: the axis tick labels use SI prefixes, so 100m means 0.1).

#### **Step Five: Shimming**

Once the centre frequency has been established and the system set-up to excite and receive signals of that frequency, the system must be shimmed in order to maximize the homogeneity of the magnetic field. Run the **Autoshim** tool below with the **Coarse** setting to shim the system. The *Coarse* setting will start by resetting all the shims to zero and uses a wide bandwidth, whereas the *Quick* setting loads the saved values and uses a narrow bandwidth; it is suitable for retuning the system once everything is set up. The shims will be saved to `shims.yaml` in your `LAB_DIR`, to be used by the FID experiment above.

This shim tool uses the Nelder-Mead simplex method to optimise the sum of squares of the FID signal by varying the current in eight shim coils designed to produce magnetic fields which are spherical harmonics; three first order ($x$, $y$, $z$) and five second order ($z^2$, $zx$, $zy$, $xy$, $x^2-y^2$).

In [ ]:
# display frequency and rf_amp again as they are used by Autoshim
pn.Row(frequency, rf_amp)

In [ ]:
from Autoshim import AutoshimApp # from dashboards-inline directory that was added to sys.path

autoshim = AutoshimApp(
    output_dir=LAB_DIR,
    override_pars=dict(
        f=frequency, # use value of frequency from the user input
        a_90=rf_amp, # use rf amplitude from the user input
        t_90=32e-6,
    )
)
autoshim.main()

#### **Step Six: Final Check**
Rerun the FID experiment using the tool below and make note of how the amplitude and decay of the signal has changed compared to without shims in **Step Three**. With a good shim there should still be significant signal at 5 ms.

In [ ]:
# create FID experiment using user shims, frequency, and RF amplitude
fid2 = FIDApp(
    override_par_files=[
        SHIM_FILE
    ],
    override_pars=dict(
        f=frequency, # use value of frequency from the user input
        a_90=rf_amp, # use rf amplitude from the user input
        t_90=32e-6,
        n_scans=1,
        n_samples=10000,
        t_dw=0.5e-6,
        t_acqdelay=50e-6,
        t_end=0.5
    )
)
fid2.main()